In [ ]:
import os
from os.path import exists
from PyPDF2 import PdfFileWriter, PdfFileReader
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import io
import gspread
from reportlab.lib.units import inch, mm, cm, pica
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint
from reportlab.pdfgen import canvas
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from reportlab.lib.pagesizes import letter
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import inch
from reportlab.lib.units import cm
from reportlab.lib.pagesizes import letter
from datetime import *
from googleapiclient import discovery
from apiclient.http import MediaFileUpload
if os.path.isdir('output'):
    pass
else:
    path = os.path.join("output")
    os.mkdir(path)
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive","https://www.googleapis.com/auth/gmail.send"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credsjer.json", scope)
client = gspread.authorize(creds)
import time
time.sleep(4)
try:
    creds = ServiceAccountCredentials.from_json_keyfile_name("credsjer.json", scope)
    client = gspread.authorize(creds)
    sheet = client.open("Form Automation to PDF").sheet1  # Open the spreadhseet
except:
    time.sleep(10)
    creds = ServiceAccountCredentials.from_json_keyfile_name("credsjer.json", scope)
    client = gspread.authorize(creds)
    sheet = client.open("Form Automation to PDF").sheet1  # Open the spreadhseet
from datetime import *
timestmp=sheet.col_values(1)
fullname=sheet.col_values(2)
bd=sheet.col_values(3)
age=sheet.col_values(7)
gender=sheet.col_values(4)
date_of_appointment=sheet.col_values(5)
hour_of_appointment=sheet.col_values(6)
del fullname[0]
del bd[0]
del gender[0]
del date_of_appointment[0]
del hour_of_appointment[0]
del timestmp[0]
with open("txt.txt","r") as x:
    p=x.readline()
p=p.replace("\n","")
for i in range(len(fullname)):
    if os.path.exists(f'{fullname[i]}.pdf'):
        p=p+1
        pass
    else:
        packet = io.BytesIO()
        pdfmetrics.registerFont(TTFont('Vera', 'Vera.ttf'))
        can = canvas.Canvas(packet, pagesize=letter)
        can.setFont('Vera', 8)
        can.drawString(405, 682, '{0:05}'.format(int(p)))
        name=can.drawString(93, 679, fullname[i])
        patient=can.drawString(93, 668, '{0:05}'.format(int(p)))
        birthday=can.drawString(270, 670, bd[i])
        birthday=bd[i].split("/")
        pl=date(int(birthday[2]),int(birthday[0]),int(birthday[1]))
        today=date.today()
        t=today.year-pl.year-((today.month, today.day)<(int(birthday[0]),int(birthday[1])))
        can.drawString(270, 657, f"{t} years")
        can.drawString(270, 644, gender[i])
        o=datetime.strptime(hour_of_appointment[i],"%I:%M:%S %p")
        hour_times=o.strftime("%H:%M:%S")
        hour_time= datetime.strptime(hour_times, "%H:%M:%S")
        hour_times= datetime.strptime(hour_times, "%H:%M:%S")
        hour_times=hour_times.strftime("%I:%M %p")
        two_hours = hour_time + timedelta(hours=2)
        dot=two_hours.strftime("%I:%M %p")
        can.drawString(425, 657, f"{date_of_appointment[i]} {hour_times}")
        width = can._pagesize[0]
        padding =  12* mm
        can.drawRightString(width-padding, 50, f"Lab results: {fullname[i]}")
        can.drawRightString(width-padding+3, 62.5, f"Accession: {'{0:04}'.format(int(p))} Patient #: {'{0:05}'.format(int(p))} ")
        can.drawString(125, 62, f"{date_of_appointment[i]} {hour_times}")
        can.drawString(70, 50, f"{date_of_appointment[i]} {dot}")
        can.save()
        #move to the beginning of the StringIO buffer
        packet.seek(0)

        # create a new PDF with Reportlab
        new_pdf = PdfFileReader(packet)
        # read your existing PDF
        existing_pdf = PdfFileReader(open("Full Name (2).pdf", "rb"))
        output = PdfFileWriter()
        # add the "watermark" (which is the new pdf) on the existing page
        page = existing_pdf.getPage(0)
        page.mergePage(new_pdf.getPage(0))
        output.addPage(page)
        # finally, write "output" to a realµ file
        outputStream = open(f"output/{fullname[i]}.pdf", "wb")
        output.write(outputStream)
        outputStream.close()
        p=int(p)+1
with open("txt.txt","w") as x:
    x.write(str(p))
sheet.clear()
cell_list = sheet.range('A1:F1')
cell_values = ["Timestamp","Full Name","Birthdate","Gender","Date of Appointment","Time of the appointment"]

for i, val in enumerate(cell_values):  #gives us a tuple of an index and value
    cell_list[i].value = val    #use the index on cell_list and the val from cell_values

sheet.update_cells(cell_list)
quit()
